# Indexing
在这里，我们将查看使用LangChain索引API的基本索引工作流。

索引API允许您将来自任何源的文档加载到矢量存储中并保持同步。具体来说，它很有帮助
- 避免将重复的内容写入vector存储
- 避免重写未更改的内容
- 避免在未更改的内容上重新计算嵌入

所有这些都可以节省你的时间和金钱，并改善你的矢量搜索结果。       
至关重要的是，索引API甚至可以处理相对于原始源文档已经经历了几个转换步骤(例如，通过文本分块)的文档。


## How it works
LangChain索引使用记录管理器(`RecordManager`)来跟踪写入矢量存储的文档。

索引内容时，计算每个文档的哈希值，并将以下信息存储在记录管理器中
- 文档散列(页面内容和元数据的散列)
- 写入时间
- 每个文档的源id应该在其元数据中包含信息，以便我们确定该文档的最终来源



## 删除模式
将文档索引到矢量存储时，可能会删除矢量存储中的一些现有文档。在某些情况下，您可能希望删除与正在索引的新文档来自相同来源的所有现有文档。在其他情况下，您可能希望批量删除所有现有文档。索引API删除模式允许您选择所需的行为
表格
| 清理方式 | 减少重复内容 | 可并行性 | 清理已删除的源文档 | 清理源文档和/或派生文档的突变 | 清理时间 |
| ----------- | ----------- |----------- |----------- |----------- |----------- |
| None | ✅ | ✅ | ❌ | ❌ | - |
| 增量式 | ✅ | ✅ | ❌ | ✅ | 连续地 |
| 完整 | ✅ | ❌ | ✅ | ✅ | 索引结束时 |

None不做任何自动清理，允许用户手动清理旧内容。
增量和完整提供以下自动清理：
- 如果源文档或派生文档的内容发生了变化，增量模式或完整模式都将清除(删除)以前版本的内容。
- 如果源文档已被删除(意味着它不包括在当前索引的文档中)，则完全清理模式将正确地从矢量存储中删除它，但增量模式不会。

当内容发生变化(例如，源PDF文件被修改)时，在索引期间会有一段时间，新旧版本都可能返回给用户。这发生在新内容写入之后，但在旧版本被删除之前。
- 增量 索引最大限度地减少了这段时间，因为它能够在写入时连续地进行清理。
- 完整 模式在所有批写入后进行清理。

**要求**
1. 不要与预先填充了独立于索引API的内容的存储一起使用，因为记录管理器将不知道以前已经插入了记录。  
2. 仅支持LangChain `vectorstore`   
   1. 通过id添加文档(添加带有ids参数的文档方法)   
   2. 按id删除(带id参数的删除方法)  


兼容Vectorstores:AnalyticDB, AstraDB, AzureCosmosDBVectorSearch, AzureSearch, AwaDB, Bagel, Cassandra, Chroma, CouchbaseVectorStore, DashVector, DatabricksVectorSearch, DeepLake, Dingo, ElasticVectorSearch, ElasticsearchStore, FAISS, HanaDB, LanceDB, Milvus, MyScale, OpenSearchVectorSearch, PGVector, Pinecone, Qdrant, Redis, Rockset, ScaNN, SupabaseVectorStore, SurrealDBStore, TimescaleVector, UpstashVectorStore, Vald, VDMS, Vearch, VespaStore, Weaviate, ZepVectorStore, TencentVectorDB, OpenSearchVectorSearch。

### 警告
记录管理器依赖于基于时间的机制来确定哪些内容可以被清理(当使用完全或增量清理模式时)。

如果两个任务连续运行，并且第一个任务在时钟时间改变之前完成，那么第二个任务可能无法清理内容。

由于以下原因，这在实际设置中不太可能成为问题
- RecordManager使用更高分辨率的时间戳。
- 在第一个任务和第二个任务运行之间需要更改数据，如果任务之间的时间间隔很短，则不太可能更改数据。
- 索引任务通常需要几毫秒以上的时间。




In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug
import os
load_dotenv(find_dotenv())
set_debug(False)

## Quickstart

In [2]:
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

初始化矢量存储并设置嵌入

In [6]:
collection_name = "test_index"

embedding = OpenAIEmbeddings()

vectorstore = ElasticsearchStore(
    es_url="http://192.168.26.200:9200", index_name="test_index", embedding=embedding
)

Error connecting to Elasticsearch: The client noticed that the server is not Elasticsearch and we do not support this unknown product


UnsupportedProductError: The client noticed that the server is not Elasticsearch and we do not support this unknown product

用适当的名称空间初始化记录管理器。


In [ ]:
namespace = f"elasticsearch/{collection_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)

在使用记录管理器之前创建一个模式。

In [ ]:
record_manager.create_schema()

让我们索引一些测试文档


In [ ]:
doc1 = Document(page_content="kitty", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="doggy", metadata={"source": "doggy.txt"})

索引到空vector存储区

In [ ]:
def _clear():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

`None`删除模式    
这种模式不会自动清理旧版本的内容;但是，它仍然负责内容重复删除。

In [ ]:
_clear()

In [ ]:
index(
    [doc1, doc1, doc1, doc1, doc1],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source",
)

In [ ]:
_clear()

In [ ]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

第二次将跳过所有内容

In [ ]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

`incremental`删除模式


In [ ]:
_clear()

In [ ]:
index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

再次索引将导致两个文档都被跳过，同时也会跳过嵌入操作

In [ ]:
index([], record_manager, vectorstore, cleanup="incremental", source_id_key="source")

如果我们改变一个文档，新版本将被写入，所有共享同一来源的旧版本将被删除。

In [ ]:
changed_doc_2 = Document(page_content="puppy", metadata={"source": "doggy.txt"})

In [ ]:
index(
    [changed_doc_2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

**full删除模式**

在full模式下，用户应该将需要索引的全部内容传递给索引函数。
任何没有传递给索引函数并且存在于vectorstore中的文档都将被删除
此行为对于处理源文档的删除非常有用。

In [ ]:
_clear()
all_docs = [doc1, doc2]
index(all_docs, record_manager, vectorstore, cleanup="full", source_id_key="source")

假设有人删除了第一个文档

In [ ]:
del all_docs[0]
all_docs

index(all_docs, record_manager, vectorstore, cleanup="full", source_id_key="source")

## Source
元数据属性包含一个名为source的字段。该来源应该指向与给定文档相关的最终来源。

例如，如果这些文档表示某个父文档的块，则两个文档的源应该相同并引用父文档。

通常，应该始终指定源。如果您从不打算使用增量模式，并且由于某种原因无法正确指定源字段，则仅使用None。

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
doc1 = Document(
    page_content="kitty kitty kitty kitty kitty", metadata={"source": "kitty.txt"}
)
doc2 = Document(page_content="doggy doggy the doggy", metadata={"source": "doggy.txt"})
new_docs = CharacterTextSplitter(
    separator="t", keep_separator=True, chunk_size=12, chunk_overlap=2
).split_documents([doc1, doc2])
new_docs

In [ ]:
_clear()

index(
    new_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
changed_doggy_docs = [
    Document(page_content="woof woof", metadata={"source": "doggy.txt"}),
    Document(page_content="woof woof woof", metadata={"source": "doggy.txt"}),
]

In [ ]:
index(
    changed_doggy_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
vectorstore.similarity_search("dog", k=30)

## Using with loaders
索引可以接受文档的可迭代对象或其他任何加载器。

注意:加载程序必须正确设置源密钥。

In [ ]:
from langchain_community.document_loaders.base import BaseLoader


class MyCustomLoader(BaseLoader):
    def lazy_load(self):
        text_splitter = CharacterTextSplitter(
            separator="t", keep_separator=True, chunk_size=12, chunk_overlap=2
        )
        docs = [
            Document(page_content="woof woof", metadata={"source": "doggy.txt"}),
            Document(page_content="woof woof woof", metadata={"source": "doggy.txt"}),
        ]
        yield from text_splitter.split_documents(docs)

    def load(self):
        return list(self.lazy_load())

In [ ]:
_clear()

loader = MyCustomLoader()

loader.load()

index(loader, record_manager, vectorstore, cleanup="full", source_id_key="source")

vectorstore.similarity_search("dog", k=30)